<a href="https://colab.research.google.com/github/tx1103mark/tweet-sentiment/blob/master/TPUs_in_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TPUs in Colab&nbsp; <a href="https://cloud.google.com/tpu/"><img valign="middle" src="https://raw.githubusercontent.com/GoogleCloudPlatform/tensorflow-without-a-phd/master/tensorflow-rl-pong/images/tpu-hexagon.png" width="50"></a>
In this example, we'll work through training a model to classify images of
flowers on Google's lightning-fast Cloud TPUs. Our model will take as input a photo of a flower and return whether it is a daisy, dandelion, rose, sunflower, or tulip.

We use the Keras framework, new to TPUs in TF 2.1.0. Adapted from [this notebook](https://colab.research.google.com/github/GoogleCloudPlatform/training-data-analyst/blob/master/courses/fast-and-lean-data-science/07_Keras_Flowers_TPU_xception_fine_tuned_best.ipynb) by [Martin Gorner](https://twitter.com/martin_gorner).

#### License

Copyright 2019-2020 Google LLC

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.


---


This is not an official Google product but sample code provided for an educational purpose.


## Enabling and testing the TPU

First, you'll need to enable TPUs for the notebook:

- Navigate to Edit→Notebook Settings
- select TPU from the Hardware Accelerator drop-down

Next, we'll check that we can connect to the TPU:

#Data process

In [ ]:
From 1711db7b42bc83b2f326cb431726b886a69a2503 Mon Sep 17 00:00:00 2001
From: zhengjun10 <zhengjun10@huawei.com>
Date: Sat, 3 Jul 2021 15:46:14 +0800
Subject: [PATCH] Fix lite inference jni bug

---
 build.sh                                           | 15 +--------
 mindspore/lite/examples/train_lenet/README.md      |  1 -
 mindspore/lite/examples/train_lenet/README_CN.md   |  1 -
 .../java/com/mindspore/lite/train_lenet/Main.java  |  2 +-
 .../lite/examples/transfer_learning/README.md      |  1 -
 .../lite/examples/transfer_learning/README_CN.md   |  1 -
 .../main/java/com/mindspore/lite/LiteSession.java  | 21 ++++++------
 .../main/java/com.mindspore.lite/LiteSession.java  | 25 +++++++-------
 mindspore/lite/java/native/CMakeLists.txt          |  6 ++--
 .../lite/java/native/runtime/lite_session.cpp      | 24 +++++++++++--
 .../lite/java/native/runtime/train_session.cpp     | 39 ----------------------
 11 files changed, 47 insertions(+), 89 deletions(-)
 delete mode 100644 mindspore/lite/java/native/runtime/train_session.cpp

diff --git a/build.sh b/build.sh
index a4c1039..ac91ebd 100755
--- a/build.sh
+++ b/build.sh
@@ -526,11 +526,6 @@ build_lite_x86_64_jni_and_jar()
     cp ./libmindspore-lite-jni.so ${LITE_JAVA_PATH}/java/linux_x86/libs/
     cp ./libmindspore-lite-jni.so ${LITE_JAVA_PATH}/native/libs/linux_x86/
     cp ./libmindspore-lite-jni.so ${BASEPATH}/output/tmp/${pkg_name}/runtime/lib/
-    if [[ "X$is_train" = "Xon" ]]; then
-        cp ./libmindspore-lite-train-jni.so ${LITE_JAVA_PATH}/java/linux_x86/libs/
-        cp ./libmindspore-lite-train-jni.so ${LITE_JAVA_PATH}/native/libs/linux_x86/
-        cp ./libmindspore-lite-train-jni.so ${BASEPATH}/output/tmp/${pkg_name}/runtime/lib/
-    fi
 
     # build java common
     cd ${LITE_JAVA_PATH}/java/common
@@ -715,10 +710,6 @@ build_lite_arm64_and_jni() {
     fi
     cp ./libmindspore-lite-jni.so ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/
     cp ./libmindspore-lite-jni.so ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
-    if [[ "X$is_train" = "Xon" ]]; then
-      cp ./libmindspore-lite-train-jni.so ${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/
-      cp ./libmindspore-lite-train-jni.so ${LITE_JAVA_PATH}/native/libs/arm64-v8a/
-    fi
 }
 
 build_lite_arm32_and_jni() {
@@ -760,10 +751,6 @@ build_lite_arm32_and_jni() {
     fi
     cp ./libmindspore-lite-jni.so ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/
     cp ./libmindspore-lite-jni.so ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
-    if [[ "X$is_train" = "Xon" ]]; then
-      cp ./libmindspore-lite-train-jni.so ${LITE_JAVA_PATH}/java/app/libs/armeabi-v7a/
-      cp ./libmindspore-lite-train-jni.so ${LITE_JAVA_PATH}/native/libs/armeabi-v7a/
-    fi
 }
 
 check_java_home() {
@@ -794,7 +781,7 @@ build_aar() {
 
     # build java fl_client
     local is_train=on
-    local train_so=${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/libmindspore-lite-train-jni.so
+    local train_so=${LITE_JAVA_PATH}/java/app/libs/arm64-v8a/libmindspore-lite-train.so
     if [ ! -f "$train_so" ]; then
       echo "not exist"
       is_train=off
diff --git a/mindspore/lite/examples/train_lenet/README.md b/mindspore/lite/examples/train_lenet/README.md
index a57df28..7fceab7 100644
--- a/mindspore/lite/examples/train_lenet/README.md
+++ b/mindspore/lite/examples/train_lenet/README.md
@@ -135,7 +135,6 @@ When the `prepare_and_run.sh` script is run, the following folder is prepared. I
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite-jni.so
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite.so
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train.a
-鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train-jni.so
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train.so
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libturbojpeg.so.0
 鈹偮犅?鈹偮犅?鈹斺攢鈹€ mindspore-lite-java.jar
diff --git a/mindspore/lite/examples/train_lenet/README_CN.md b/mindspore/lite/examples/train_lenet/README_CN.md
index 84d0be7..e6f78ee 100644
--- a/mindspore/lite/examples/train_lenet/README_CN.md
+++ b/mindspore/lite/examples/train_lenet/README_CN.md
@@ -124,7 +124,6 @@ train_lenet/
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite-jni.so
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite.so
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train.a
-鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train-jni.so
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train.so
 鈹偮犅?鈹偮犅?鈹溾攢鈹€ libturbojpeg.so.0
 鈹偮犅?鈹偮犅?鈹斺攢鈹€ mindspore-lite-java.jar
diff --git a/mindspore/lite/examples/train_lenet_java/src/main/java/com/mindspore/lite/train_lenet/Main.java b/mindspore/lite/examples/train_lenet_java/src/main/java/com/mindspore/lite/train_lenet/Main.java
index 39e2fd5..57ae0fe 100644
--- a/mindspore/lite/examples/train_lenet_java/src/main/java/com/mindspore/lite/train_lenet/Main.java
+++ b/mindspore/lite/examples/train_lenet_java/src/main/java/com/mindspore/lite/train_lenet/Main.java
@@ -20,7 +20,7 @@ import com.mindspore.lite.Version;
 
 public class Main {
     public static void main(String[] args) {
-        System.loadLibrary("mindspore-lite-train-jni");
+        System.loadLibrary("mindspore-lite-jni");
         System.out.println(Version.version());
         if (args.length < 2) {
             System.err.println("model path and dataset path must be provided.");
diff --git a/mindspore/lite/examples/transfer_learning/README.md b/mindspore/lite/examples/transfer_learning/README.md
index 00a2b17..8ebcf24 100644
--- a/mindspore/lite/examples/transfer_learning/README.md
+++ b/mindspore/lite/examples/transfer_learning/README.md
@@ -150,7 +150,6 @@ package-arm64/
 鈹偮犅?鈹溾攢鈹€ libmindspore-lite-jni.so
 鈹偮犅?鈹溾攢鈹€ libmindspore-lite.so
 鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train.a
-鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train-jni.so
 鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train.so
 鈹偮犅?鈹溾攢鈹€ libturbojpeg.so.0
 鈹偮犅?鈹斺攢鈹€ mindspore-lite-java.jar
diff --git a/mindspore/lite/examples/transfer_learning/README_CN.md b/mindspore/lite/examples/transfer_learning/README_CN.md
index e1441d3..8d6185f 100644
--- a/mindspore/lite/examples/transfer_learning/README_CN.md
+++ b/mindspore/lite/examples/transfer_learning/README_CN.md
@@ -135,7 +135,6 @@ package-arm64/
 鈹偮犅?鈹溾攢鈹€ libmindspore-lite-jni.so
 鈹偮犅?鈹溾攢鈹€ libmindspore-lite.so
 鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train.a
-鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train-jni.so
 鈹偮犅?鈹溾攢鈹€ libmindspore-lite-train.so
 鈹偮犅?鈹溾攢鈹€ libturbojpeg.so.0
 鈹偮犅?鈹斺攢鈹€ mindspore-lite-java.jar
diff --git a/mindspore/lite/java/java/app/src/main/java/com/mindspore/lite/LiteSession.java b/mindspore/lite/java/java/app/src/main/java/com/mindspore/lite/LiteSession.java
index da72713..331da09 100644
--- a/mindspore/lite/java/java/app/src/main/java/com/mindspore/lite/LiteSession.java
+++ b/mindspore/lite/java/java/app/src/main/java/com/mindspore/lite/LiteSession.java
@@ -28,7 +28,6 @@ import com.mindspore.lite.config.MSConfig;
 public class LiteSession {
     static {
         System.loadLibrary("mindspore-lite-jni");
-        System.loadLibrary("mindspore-lite-train-jni");
     }
 
     private long sessionPtr = 0;
@@ -64,9 +63,9 @@ public class LiteSession {
         }
     }
 
-    public static LiteSession createTrainSession(String modelname, final MSConfig config, boolean train_mode) {
+    public static LiteSession createTrainSession(String modelName, final MSConfig config, boolean trainMode) {
         LiteSession liteSession = new LiteSession();
-        liteSession.sessionPtr = liteSession.createTrainSession(modelname, config.getMSConfigPtr(), train_mode, 0);
+        liteSession.sessionPtr = liteSession.createTrainSession(modelName, config.getMSConfigPtr(), trainMode, 0);
         if (liteSession.sessionPtr == 0) {
             return null;
         } else {
@@ -78,8 +77,8 @@ public class LiteSession {
         return sessionPtr;
     }
 
-    public void bindThread(boolean if_bind) {
-        this.bindThread(this.sessionPtr, if_bind);
+    public void bindThread(boolean ifBind) {
+        this.bindThread(this.sessionPtr, ifBind);
     }
 
     public boolean compileGraph(Model model) {
@@ -156,8 +155,8 @@ public class LiteSession {
         return this.resize(this.sessionPtr, inputsArray, dims);
     }
 
-    public boolean export(String modelFilename, int model_type, int quantization_type) {
-        return this.export(this.sessionPtr, modelFilename, model_type, quantization_type);
+    public boolean export(String modelFileName, int modelType, int quantizationType) {
+        return this.export(this.sessionPtr, modelFileName, modelType, quantizationType);
     }
 
     public boolean train() {
@@ -205,11 +204,11 @@ public class LiteSession {
 
     private native long createSessionWithModel(MappedByteBuffer buffer, long msConfigPtr);
 
-    private native long createTrainSession(String filename, long msContextPtr, boolean train_mode, long msTrainCfgPtr);
+    private native long createTrainSession(String filename, long msContextPtr, boolean trainMode, long msTrainCfgPtr);
 
     private native boolean compileGraph(long sessionPtr, long modelPtr);
 
-    private native void bindThread(long sessionPtr, boolean if_bind);
+    private native void bindThread(long sessionPtr, boolean ifBind);
 
     private native boolean runGraph(long sessionPtr);
 
@@ -229,7 +228,7 @@ public class LiteSession {
 
     private native boolean resize(long sessionPtr, long[] inputs, int[][] dims);
 
-    private native boolean export(long sessionPtr, String modelFilename, int model_type, int quantization_type);
+    private native boolean export(long sessionPtr, String modelFileName, int modelType, int quantizationType);
 
     private native boolean train(long sessionPtr);
 
@@ -239,7 +238,7 @@ public class LiteSession {
 
     private native boolean isEval(long sessionPtr);
 
-    private native boolean setLearningRate(long sessionPtr, float learning_rate);
+    private native boolean setLearningRate(long sessionPtr, float learningRate);
 
     private native boolean setupVirtualBatch(long sessionPtr, int virtualBatchMultiplier, float learningRate, float momentum);
 
diff --git a/mindspore/lite/java/java/linux_x86/src/main/java/com.mindspore.lite/LiteSession.java b/mindspore/lite/java/java/linux_x86/src/main/java/com.mindspore.lite/LiteSession.java
index da72713..a49e940 100644
--- a/mindspore/lite/java/java/linux_x86/src/main/java/com.mindspore.lite/LiteSession.java
+++ b/mindspore/lite/java/java/linux_x86/src/main/java/com.mindspore.lite/LiteSession.java
@@ -28,7 +28,6 @@ import com.mindspore.lite.config.MSConfig;
 public class LiteSession {
     static {
         System.loadLibrary("mindspore-lite-jni");
-        System.loadLibrary("mindspore-lite-train-jni");
     }
 
     private long sessionPtr = 0;
@@ -64,9 +63,9 @@ public class LiteSession {
         }
     }
 
-    public static LiteSession createTrainSession(String modelname, final MSConfig config, boolean train_mode) {
+    public static LiteSession createTrainSession(String modelName, final MSConfig config, boolean trainMode) {
         LiteSession liteSession = new LiteSession();
-        liteSession.sessionPtr = liteSession.createTrainSession(modelname, config.getMSConfigPtr(), train_mode, 0);
+        liteSession.sessionPtr = liteSession.createTrainSession(modelName, config.getMSConfigPtr(), trainMode, 0);
         if (liteSession.sessionPtr == 0) {
             return null;
         } else {
@@ -78,8 +77,8 @@ public class LiteSession {
         return sessionPtr;
     }
 
-    public void bindThread(boolean if_bind) {
-        this.bindThread(this.sessionPtr, if_bind);
+    public void bindThread(boolean ifBind) {
+        this.bindThread(this.sessionPtr, ifBind);
     }
 
     public boolean compileGraph(Model model) {
@@ -156,8 +155,8 @@ public class LiteSession {
         return this.resize(this.sessionPtr, inputsArray, dims);
     }
 
-    public boolean export(String modelFilename, int model_type, int quantization_type) {
-        return this.export(this.sessionPtr, modelFilename, model_type, quantization_type);
+    public boolean export(String modelFileName, int modelType, int quantizationType) {
+        return this.export(this.sessionPtr, modelFileName, modelType, quantizationType);
     }
 
     public boolean train() {
@@ -176,8 +175,8 @@ public class LiteSession {
         return this.isEval(this.sessionPtr);
     }
 
-    public boolean setLearningRate(float learning_rate) {
-        return this.setLearningRate(this.sessionPtr, learning_rate);
+    public boolean setLearningRate(float learningRate) {
+        return this.setLearningRate(this.sessionPtr, learningRate);
     }
 
     public boolean setupVirtualBatch(int virtualBatchMultiplier, float learningRate, float momentum) {
@@ -205,11 +204,11 @@ public class LiteSession {
 
     private native long createSessionWithModel(MappedByteBuffer buffer, long msConfigPtr);
 
-    private native long createTrainSession(String filename, long msContextPtr, boolean train_mode, long msTrainCfgPtr);
+    private native long createTrainSession(String fileName, long msContextPtr, boolean trainMode, long msTrainCfgPtr);
 
     private native boolean compileGraph(long sessionPtr, long modelPtr);
 
-    private native void bindThread(long sessionPtr, boolean if_bind);
+    private native void bindThread(long sessionPtr, boolean ifBind);
 
     private native boolean runGraph(long sessionPtr);
 
@@ -229,7 +228,7 @@ public class LiteSession {
 
     private native boolean resize(long sessionPtr, long[] inputs, int[][] dims);
 
-    private native boolean export(long sessionPtr, String modelFilename, int model_type, int quantization_type);
+    private native boolean export(long sessionPtr, String modelFilename, int modelType, int quantizationType);
 
     private native boolean train(long sessionPtr);
 
@@ -239,7 +238,7 @@ public class LiteSession {
 
     private native boolean isEval(long sessionPtr);
 
-    private native boolean setLearningRate(long sessionPtr, float learning_rate);
+    private native boolean setLearningRate(long sessionPtr, float learningRate);
 
     private native boolean setupVirtualBatch(long sessionPtr, int virtualBatchMultiplier, float learningRate, float momentum);
 
diff --git a/mindspore/lite/java/native/CMakeLists.txt b/mindspore/lite/java/native/CMakeLists.txt
index 239005f..2353b1d 100644
--- a/mindspore/lite/java/native/CMakeLists.txt
+++ b/mindspore/lite/java/native/CMakeLists.txt
@@ -103,13 +103,11 @@ endif()
 
 if(SUPPORT_TRAIN)
   set(LITE_TRAIN_SO_NAME mindspore-lite-train minddata-lite)
-  set(JNI_TRAIN_SRC ${CMAKE_CURRENT_SOURCE_DIR}/runtime/train_session.cpp)
-  add_library(mindspore-lite-train-jni SHARED ${JNI_TRAIN_SRC})
   if(PLATFORM_ARM64 OR PLATFORM_ARM32)
     find_library(log-lib log)
-    target_link_libraries(mindspore-lite-train-jni ${LITE_TRAIN_SO_NAME} ${log-lib})
+    target_link_libraries(mindspore-lite-jni ${LITE_TRAIN_SO_NAME} ${log-lib})
   else()
-    target_link_libraries(mindspore-lite-train-jni ${LITE_TRAIN_SO_NAME})
+    target_link_libraries(mindspore-lite-jni ${LITE_TRAIN_SO_NAME})
   endif()
 endif()
 
diff --git a/mindspore/lite/java/native/runtime/lite_session.cpp b/mindspore/lite/java/native/runtime/lite_session.cpp
index f33a557..6abe74c 100644
--- a/mindspore/lite/java/native/runtime/lite_session.cpp
+++ b/mindspore/lite/java/native/runtime/lite_session.cpp
@@ -356,8 +356,8 @@ extern "C" JNIEXPORT jboolean JNICALL Java_com_mindspore_lite_LiteSession_setLea
 
 extern "C" JNIEXPORT jboolean JNICALL Java_com_mindspore_lite_LiteSession_setupVirtualBatch(JNIEnv *env, jobject thiz,
                                                                                            jlong session_ptr,
-                                                                                           jint virtualBatchMultiplier,
-                                                                                           jfloat learningRate,
+                                                                                           jint virtual_batch_multiplier,
+                                                                                           jfloat learning_rate,
                                                                                            jfloat momentum) {
   auto *session_pointer = reinterpret_cast<void *>(session_ptr);
   if (session_pointer == nullptr) {
@@ -365,7 +365,7 @@ extern "C" JNIEXPORT jboolean JNICALL Java_com_mindspore_lite_LiteSession_setupV
     return (jboolean) false;
   }
   auto *lite_session_ptr = static_cast<mindspore::session::LiteSession *>(session_pointer);
-  auto ret = lite_session_ptr->SetupVirtualBatch(virtualBatchMultiplier, learningRate, momentum);
+  auto ret = lite_session_ptr->SetupVirtualBatch(virtual_batch_multiplier, learning_rate, momentum);
   return (jboolean)(ret == mindspore::lite::RET_OK);
 }
 
@@ -411,3 +411,21 @@ extern "C" JNIEXPORT jobject JNICALL Java_com_mindspore_lite_LiteSession_getFeat
   }
   return ret;
 }
+
+extern "C" JNIEXPORT jlong JNICALL Java_com_mindspore_lite_LiteSession_createTrainSession(
+  JNIEnv *env, jobject thiz, jstring file_name, jlong ms_context_ptr, jboolean train_mode, jlong train_config_ptr) {
+  auto *pointer = reinterpret_cast<void *>(ms_context_ptr);
+  if (pointer == nullptr) {
+  MS_LOGE("Context pointer from java is nullptr");
+  return jlong(nullptr);
+  }
+  auto *lite_context_ptr = static_cast<mindspore::lite::Context *>(pointer);
+
+  auto session = mindspore::session::LiteSession::CreateTrainSession(env->GetStringUTFChars(file_name, JNI_FALSE),
+                                                                     lite_context_ptr, train_mode, nullptr);
+  if (session == nullptr) {
+  MS_LOGE("CreateTrainSession failed");
+  return jlong(nullptr);
+  }
+  return jlong(session);
+}
diff --git a/mindspore/lite/java/native/runtime/train_session.cpp b/mindspore/lite/java/native/runtime/train_session.cpp
deleted file mode 100644
index a838147..0000000
--- a/mindspore/lite/java/native/runtime/train_session.cpp
+++ /dev/null
@@ -1,39 +0,0 @@
-/**
- * Copyright 2020 Huawei Technologies Co., Ltd
- *
- * Licensed under the Apache License, Version 2.0 (the "License");
- * you may not use this file except in compliance with the License.
- * You may obtain a copy of the License at
- *
- * http://www.apache.org/licenses/LICENSE-2.0
- *
- * Unless required by applicable law or agreed to in writing, software
- * distributed under the License is distributed on an "AS IS" BASIS,
- * WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
- * See the License for the specific language governing permissions and
- * limitations under the License.
- */
-
-#include <jni.h>
-#include "common/ms_log.h"
-#include "include/lite_session.h"
-#include "include/train/train_cfg.h"
-#include "include/errorcode.h"
-
-extern "C" JNIEXPORT jlong JNICALL Java_com_mindspore_lite_LiteSession_createTrainSession(
-  JNIEnv *env, jobject thiz, jstring file_name, jlong ms_context_ptr, jboolean train_mode, jlong train_config_ptr) {
-  auto *pointer = reinterpret_cast<void *>(ms_context_ptr);
-  if (pointer == nullptr) {
-    MS_LOGE("Context pointer from java is nullptr");
-    return jlong(nullptr);
-  }
-  auto *lite_context_ptr = static_cast<mindspore::lite::Context *>(pointer);
-
-  auto session = mindspore::session::LiteSession::CreateTrainSession(env->GetStringUTFChars(file_name, JNI_FALSE),
-                                                                     lite_context_ptr, train_mode, nullptr);
-  if (session == nullptr) {
-    MS_LOGE("CreateSession failed");
-    return jlong(nullptr);
-  }
-  return jlong(session);
-}
-- 
2.7.4

